# N-gram Language Models


N-gram language models are a way of predicting the next word based on n-1 preceding words.

We will make our own n-gram language model from scratch, with the help of a few utilities from NLTK.

<center>
<table><tr>
<td> <img src="../img/shannon_example_b.png" alt="Drawing" style="height: 250px;"/><figcaption style="width: 280px;">A language consisting only of the letters A, B, C, D, and E. Each letter has a probability.</figcaption></td>
<td> <img src="../img/shannon_example_c.png" alt="Drawing" style="height: 250px;"/><figcaption style="width: 350px;">A language consisting only of the letters A, B, and C but with conditional transition probabilities.</figcaption></td>
</tr></table>
</center>

<center>
<table><tr>
<td> <img src="../img/onefish.png" alt="Drawing" style="height: 400px;"/><figcaption style="width: 280px;">A markov model of the sentence "one fish two fish red fish blue fish".</figcaption></td>
</tr></table>
</center>

<center>
<table><tr>
<td> <img src="../img/quicklyran.png" alt="Drawing" style="height: 400px;"/><figcaption style="width: 280px;">Markov model for generating a sentence of infinite possible length.</figcaption></td>
</tr></table>
</center>

## Order

The order of a Markov model is the number of previous states that the model takes into account. This model is second order, because we care about the previous two words when detexrmining the next word.

Let's start with a toy corpus of three sentences

In [4]:
corpus = """By this liberty they entered into a very laudable emulation to do all of them \
what they saw did please one. If any of the gallants or ladies should say, Let us drink, \
they would all drink.  If any one of them said, Let us play, they all played.  If one said, \
Let us go a-walking into the fields they went all."""

## Preprocessing

The first step of any text-based data project is usually pre-processing the text. 

**Tokenization:** Separate into contexts, words.

**(Normalization):** Multiple spellings, capitalization.

**(Lemmatization):** Different forms of a word collapse into one.

**(Stop-word removal):** Discarding common words.

**(Frequency Limits):** Discard very infrequent words.


# Exercise 

### 1: Tokenization

Tokenize and the corpus and normalize it by making all the words the same case. That is, split the corpus into tokens and store them in a list. The output should look like a list of strings, e.g.

 ```
       tokenized_corpus = ["by", "this", "liberty", ... "all", "."']
 ```

You will use the `str.split()` method, the `str.lower()` or `str.upper()` methods. 

We also want punctuation to be its own token, which will take more than just splitting on whitespace.  If you like regexes, try building one that will separate punctuation from the preceding word, so that it registers as a separate token. You can use `re.sub()` for this. Consult the Python docs (these are your friend!) for information on these functions and to look for others that might help https://docs.python.org/3.11 . No shade for asking chatGPT to do this exercise for you, but dont ask it until you are sure youre ready to see how deep the rabbit hole goes.



# Get N-grams


In [ ]:
# replace word+punctuation with word + space + punctuation
# e.g. "end." => "end ." 
# (\w) = any word character
# ([.,?!;:]) = any punctuation character
# the parentheses define 'capture groups', and \1 and \2 refer back to the first and second capture groups
tokenized_corpus = re.sub(r'(\w)([.,?!;:])', r'\1 \2', corpus) 
tokenized_corpus = tokenized_corpus.split()
tokenized_corpus = [word.lower() for word in tokenized_corpus]
print(tokenized_corpus)

<center>
<table><tr>
<td> <img src="../img/fishbigrams.png" alt="Drawing" style="height: 400px;"/><figcaption style="width: 280px;"> Bigrams for the sentence "one fish two fish red fish blue fish".</figcaption></td>
</tr></table>
</center>

<center>
<table><tr>
<td> <img src="../img/fishpairs.png" alt="Drawing" style="height: 400px;"/><figcaption style="width: 280px;">Bigrams for the sentence "one fish two fish red fish blue fish".</figcaption></td>
</tr></table>
</center>

### Exercise: Get N-grams and N-gram Frequencies

Build a bigram conditional frequency distribution for each state given the last state. For this you will need to write two functions:

   `ngrams(text,n)` should transform a text into a list of ngrams (n-token) spans in the text. For example, ngrams(tokenized_corpus, 2) would return something like :

```
    [('by', 'this'),
     ('this', 'liberty'),
     ('liberty', 'they'),
     ('they', 'entered'),
      ...
    ]
```

If you are feeling stuck, try writing the function for bigrams first. That is, write it for just 2-word window size,

   `ngram_frequency(ngram_list)` should read in the list of ngrams generated in the last step and construct a dictionary mapping from ngrams to integer counts, like:
    
```
    {('by', 'this'): 1,
     ('this', 'liberty'): 1,
     ('liberty', 'they'): 1,
      ...
    }
```

This function should be more straightforward.
    

In [ ]:
n = 2 # 2 for bigram 3 for trigram, etc
ngram_list = ngrams(tokenized_corpus, n)
ngram_list

In [ ]:
# get the ngrams for a corpus
def ngrams(text, n):
    raise Exception("not implemented")

We have generated  state/transition pairs which we formed by using a “window” to look at what the next token is in a pair. Pair are in the form
    (current state, next word)


In [ ]:
# calculate frequencies of the n-grams
def ngram_frequency(ngram_list):
    raise Exception("not implemented")

In [ ]:
ngram_frequencies = ngram_frequency(ngram_list)
ngram_frequencies

# From Frequencies to Probabilities

The next step is to take our frequency counts and convert them into probabilities.



In [10]:
# this gives you the raw probabilities 
def ngram_probs(n_gram_frequencies, n):
    probs = []
    total = len(n_gram_frequencies)
    for ngram_value, ngram_count in ngram_frequencies.items():
        prob = ngram_count / total
        probs.append([ngram_value, prob])
    return probs

# with laplace smoothing
def ngram_probs_laplace_one(ngram_frequencies, n):
    laplace_one_probs = []
    lessgram = ngram_frequency(ngrams(tokenized_corpus, n-1))
    vocabulary = len(ngram_frequencies)
    for ngram_value, ngram_count in ngram_frequencies.items():
        try:
            prob = (ngram_count + 1) / (lessgram[ngram_value[:-1]] + vocabulary)
            sum_prob = ++prob
        except KeyError:
            prob = (ngram_count + 1) / (bigram["UNK"] + vocabulary)
            sum_prob = ++math.log(prob)
        laplace_one_probs.append([ngram_value, prob])
    return laplace_one_probs

In [11]:
ngram_probs(ngram_frequencies, n)

NameError: name 'ngram_frequencies' is not defined

In [18]:
def ngrams(text, n):
    ngram_list = []
    for index, word in enumerate(text[:-n+1]):
        gram = (word, text[index + 1]) 
        ngram_list.append(gram)
    print(gram)

In [12]:
ngram_probs_laplace_one(ngram_frequencies, n)

NameError: name 'ngram_frequencies' is not defined

We've seen the word let. what is the probability that us is the next word?



it's NOT .063. why is this? 

## An aside: data structures for counting words

As an aside, first a few quick words about data structures in NLTK that support us in counting words (or word groups, or pieces of syntactic structure). The first is basically a dictionary mapping words to counts, called a FreqDist. Conveniently, you can just initialize it by giving it a list of items, and it will count how often each item appears in the list:

In [9]:
# NLTK data structures for counting stuff:
# count individual words or other items:
import nltk

fd = nltk.FreqDist(["a", "b", "c", "a", "b", "d"])
fd

FreqDist({'a': 2, 'b': 2, 'c': 1, 'd': 1})

The second data structure relevant for us today is the ConditionalFreqDist. It also has counts, but it can be used to count, for each target, how often each context word appears, or more generally, how often each word appears given some other word. Say "a" is a target, and "b" and "c" are context items, then a ConditionalFreqDist can be used like a two-deep dictionary, whose first-level keys are called "conditions":

In [23]:
# for targets, count context words,
# or in general, for one sort of items, 
# count another sort of items

cfd = nltk.ConditionalFreqDist()
cfd["a"]["b"] += 1
cfd["a"]["c"] += 1
cfd

<ConditionalFreqDist with 1 conditions>

For the "condition" 'a', the entry is again a FreqDist object that counts appearances of 'b' and 'c':

In [14]:
cfd["a"]

FreqDist({'b': 1, 'c': 1})

You can also initialize a ConditionalFreqDist by a list of pairs. It then counts, for each first item of the pair, how often each second item appears. In the next example, the ConditionalFreqDist will record that given "a", both "b" and "c" appeared once, and that given "d", "e" appeared once:

In [15]:
cfd = nltk.ConditionalFreqDist([("a", "b"), ("a", "c"), ("d", "e")])
cfd

<ConditionalFreqDist with 2 conditions>

In [16]:
cfd["a"]

FreqDist({'b': 1, 'c': 1})

# Back to our corpus
## Conditional Probability

We then organize these pairs by the current state. We match every state to the number of possible ways to transition out of that state.

<center>
<table><tr>
<td> <img src="../img/fishprobs.png" alt="Drawing" style="height: 400px;"/><figcaption style="width: 280px;">A markov model of the sentence "one fish two fish red fish blue fish".</figcaption></td>
</tr></table>
</center>

In [25]:
from nltk import ConditionalFreqDist

cfd = ConditionalFreqDist(ngram_list)
cfd.items()

NameError: name 'ngram_list' is not defined

In [20]:
from nltk.probability import ConditionalProbDist, ELEProbDist

cpd = ConditionalProbDist(cfd, ELEProbDist, 10)
cpd.items()

dict_items([('a', <ELEProbDist based on 2 samples>), ('d', <ELEProbDist based on 1 samples>)])

In [21]:
# let's look at one of the probability distributions

cpd['one'].samples()

dict_keys([])

In [28]:
# this class comes with a generate function that makes a random sample according to the probability distirbution, 
# like rolling a weighted die
cpd['one'].generate()

IndexError: Cannot choose from an empty sequence

<center>
<table><tr>
<td> <img src="../img/fishmarkov.png" alt="Drawing" style="height: 400px;"/><figcaption style="width: 280px;">A markov model of the sentence "one fish two fish red fish blue fish".</figcaption></td>
</tr></table>
</center>

<td> <img src="../img/fishmarkovweights.png" alt="Drawing" style="height: 400px;"/><figcaption style="width: 280px;">A markov model of the sentence "one fish two fish red fish blue fish".</figcaption></td>


## Exercise: text generation with an n-gram model

Write a function called generate() that generates sentences using the model. It takes as input our bigram conditional probability distribution and a 'num_sentences' parameter. It uses the `ConditionalFreqDist.generate()` method to generate one token at a time, conditioned on the last token it generated.

Questions to think about: How should each sentence start? How will you know when you are done?

In [37]:
def generate(cpdist, num_sentences=10, seed=None):
    """
    model is an nltk ConditionalProbDist
    length is the number of tokens we wish to generate.
    """

    # to add to
    string = []
    
    
    seed = cpd['.'].generate()
    string.append(seed)
    lessgram=seed
    
    
    for i in range(num_sentences):
    # start by sampling a word that comes after a period
        while True:
            next_token = cpd[lessgram].generate()
            string.append(next_token)
            lessgram = string[-1]

            if next_token == '.':
                break
    return ' '.join(string)

generate(cpd, num_sentences=3)

IndexError: Cannot choose from an empty sequence

# Don't PEEK!!!
# Don't PEEK!!!
# Don't PEEK!!!
# Don't PEEK!!!
# Don't PEEK!!!
# Don't PEEK!!!
# Don't PEEK!!!
# Don't PEEK!!!
# Don't PEEK!!!
# Don't PEEK!!!
# Don't PEEK!!!
# Don't PEEK!!!
# Don't PEEK!!!
# Don't PEEK!!!


### Putting it all together

Let's put some of the methods together in a class.

In [39]:

class BigramModel():
    from nltk import ConditionalFreqDist
    from nltk.probability import ConditionalProbDist, ELEProbDist

    
    def __init__(self, corpus):
        n = 2
        tokenized_corpus = self._tokenize(corpus)
        self._ngrams = self._build_ngrams(tokenized_corpus, n)
        self._cpd = self._build_distribution(self._ngrams, n)
        
        
        
    def _tokenize(self, corpus):
        tokenized_corpus = re.sub(r'(\w)([.,?!;:])', r'\1 \2', corpus) 
        tokenized_corpus = tokenized_corpus.split()
        tokenized_corpus = [word.lower() for word in tokenized_corpus]
        return tokenized_corpus
            
    def _build_ngrams(self, tokenized_corpus, n):
        n_grams = []
        for i in range(n-1, len(tokenized_corpus)): 
            n_grams.append(tuple(tokenized_corpus[i-(n-1):i+1]))
        return n_grams
    
    def _build_distribution(self, corpus, n):
        cfd = ConditionalFreqDist(self._ngrams)
        cpd = ConditionalProbDist(cfd, ELEProbDist, 10)
        self.cpd = cpd
        return cpd
        
        
    def generate(self, num_sentences=10, seed=None):
        string = []
        
        seed = self._cpd['.'].generate()
        string.append(seed)
        lessgram=seed

        for i in range(num_sentences):
        # start by sampling a word that comes after a period
            while True:
                next_token = self._cpd[lessgram].generate()
                string.append(next_token)
                lessgram = string[-1]

                if next_token == '.':
                    break
        return ' '.join(string)


NameError: name 'kjv' is not defined

In [36]:
model = BigramModel(corpus)

model.generate(3)

'if any of them what they entered into the gallants or ladies should say , let us go a-walking into a very laudable emulation to do all drink , they entered into the fields they went all . if any one of the fields they would all drink . if any of them said , let us play , they all played .'

# Generalizing the bigram model

The bigram model is cool, but it still generates very unusial structures like ''they would all of the fields''. While this is beautiful, we might be after something a little closer to 'passing' as language. 

This can be achieved by conditioning on more than one previous word at a time. So, instead of just seeing 'of', the model maintains state for the last two tokens seen: 'all of'. This reduces the number of 

In fact, it would be great if we could generalize our model for arbitrary n.



# Exercise

Write a function that will build an n-gram conditional probability distribution.

**Hint:** the ConditionalFrequencyDistribution class can also be instantiated empty like this

`cfd = ConditionalFreqDist()`

You can add frequencies for conditions by accessing them like a dictionary, and they'll be created automatically if they don't already exist.
Like this: 

`cfd["a"]["b"] += 1`

This will add one to the frequency count for the b outcome of the a condition. Conditions don't have to be strings but can be more complex objects like tuples.


In [ ]:
 def _build_distribution(self, corpus, n):
    # we have to transform the ngram list to be priors and possible outcomes.
    # how do you split up a trigram? a 4gram?
    cpd = None
    self.cpd = cpd
    return cpd

Now we can rewrite the BigramModel class to construct an n-gram model for an arbitrary n. The class constructor takes n as an input in addition to the training corpus

In [ ]:
from nltk.lm.preprocessing import pad_both_ends
from nltk import ConditionalFreqDist
from nltk.probability import ConditionalProbDist, ELEProbDist
from nltk.lm.preprocessing import pad_both_ends


class NgramModel():

    
    def __init__(self, corpus, n):
        tokenized_corpus = self._tokenize(corpus)
        self.n = n
        self._ngrams = self._build_ngrams(tokenized_corpus, n)
        self._cpd = self._build_distribution(self._ngrams, n)
        
        
        
    def _tokenize(self, corpus):
        tokenized_corpus = re.sub(r'(\w)([.,?!;:])', r'\1 \2', corpus) 
        tokenized_corpus = tokenized_corpus.split()
        tokenized_corpus = [word.lower() for word in tokenized_corpus]
        
        # add pad tokens to the corpus
        tokenized_corpus = list(pad_both_ends(tokenized_corpus, n=2))
        return tokenized_corpus
            
    def _build_ngrams(self, tokenized_corpus, n):
        n_grams = []
        for i in range(n-1, len(tokenized_corpus)): 
            n_grams.append(tuple(tokenized_corpus[i-(n-1):i+1]))
        return n_grams
    
        print(n_grams)
        return n_grams
    
    def _build_distribution(self, corpus, n):
        raise Exception("not implemented")
        
        self.cpd = None
        return cpd
        
        
    def generate(self, num_tokens = 20, seed = []):
        """
        There are two cases to deal with here. Either we have a start string, or we don't. 
        If we are given a start string, we'll have to find the last n-1 gram and condition on that
        If we are not, we need to generate the first n-1 gram.
        """
        string = []
        
    
        seed = self.cpd['.'].generate()
        string.append(seed)
        lessgram=seed

        for i in range(num_tokens):
        # start by sampling a word that comes after a period
            next_token = cpdist[lessgram].generate()
            string.append(next_token)
            lessgram = string[-1]
            
        return ' '.join(string)

In [ ]:
model = NgramModel(corpus, 3)
model._cpd.items()

In [ ]:
model = NgramModel(corpus, 4)
model._cpd.items()

But there is a problem. The generate function won't work! We need to extend it to work for arbitrary n

## Exercise (if time)

Implement a generate function for our arbitrary n-gram model class. It should take as input a keyword argument `num_sentences` with the number of sentences to generate. Optionally, you can write it with a `seed` keyword argument that has an initial input to the model. If no input is given, the model can't condition on the previous n-1 grams! What should it start with? One trick for dealing with this is to take the input string and append start tokens to the front of it, so there are always enough tokens to condition on. You can use the `pad_sequence` function from NLTK for this. Otherwise, you will have to deal with the start case separately.

In [ ]:
from nltk.util import pad_sequence


def generate(self, num_sentences = 1, seed = []):
    """
    There are two cases to deal with here. Either we have a start string, or we don't. 
    If we are given a start string, we'll have to find the last n-1 gram and condition on that
    If we are not, we need to generate the first n-1 gram. For a trigram model, we need a bigram. But how can we use our model to generate new words when we have fewer than two words to condition on?
    We can use a bigram model! But wait. If we have a bigram model, how do we generate the first token without another token to condition on? 
    We can use a unigram model! 
    Recursion will save us here. Turns out the easiest way to do this will be to recursively construct an n-1gram model and store it in the main model.
    And how can we 
    Either way, we need a seed condition to enter into the loop with.
    """

    # place to put generated tokens
    string = []

    if seed:
        string = string + (list(pad_sequence(seed, self.n, pad_left=True, pad_right=False, left_pad_symbol='<s>') ) )
    else:
        string = string + (list(pad_sequence('', self.n, pad_left=True, pad_right=False, left_pad_symbol='<s>') ) )

    for i in range(num_sentences):
        next_token = tuple(string[-(self.n-1):])

        # keep generating tokens as long as we havent reached the stop sequence
        while next_token != '</s>':

            # get the last n-1 tokens to condition on next
            lessgram = tuple(string[-(self.n-1):])


            next_token = self.cpd[lessgram].generate()
            string.append( next_token )

    string = ' '.join(string)

    return string


In [ ]:
model = NgramModel(corpus, 3)

#print(model.cpd.items())
#print(model.cpd[('<s>', '<s>')].generate())

model.generate(5)